Importing all dependencies

In [1]:
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.INFO) #Provide use full information
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout)) # Give messages that helps in monitoring
from llama_index import (SimpleDirectoryReader,# To Read Unstructured data
                         LLMPredictor,#Generating Predictions using LLM
                         ServiceContext,#Supplies Contextual Data 
                         KnowledgeGraphIndex)#Both Construction and manipulation of of KG
#
from llama_index.graph_stores import SimpleGraphStore #For storing Graph data
from llama_index.storage.storage_context import StorageContext
from llama_index.llms import HuggingFaceInferenceAPI # Module to leverage Open source LLMs
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from llama_index.embeddings import LangchainEmbedding #For Embeddings
from pyvis.network import Network

Loading Open Source HuggingFace Models